In [1]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

In [2]:
if False:
    !rm ./../logs/*

# -1) Cycliste

In [3]:
class logger():
    """
    Save data into memory (in an array) and write it down to a file after [nb_log] is reached.
    """
    def __init__(self, path, limit_to_save_to_disk=25, header=None):
        import os.path
        self.path    = path
        self.limit   = limit_to_save_to_disk
        self.logs    = []
        self.nb_logs = 0
        if header:
            if not os.path.isfile(path):
                with open(path, "a") as f:
                    f.write(header)
                    f.write("\n")
        
    def log(self, string):
        self.logs.append(string)
        self.nb_logs = self.nb_logs +1
        if self.limit < self.nb_logs:
            self.write_to_disk()
    
    def write_to_disk(self):
        from time import time
        heure = time()
        new_path = "%s_%s.csv"%(self.path, heure)
        new_path = self.path
        with open(new_path, "a") as f:
            while len(self.logs):
                try:
                    a = self.logs.pop()
                    f.write(u"%s\n"%a.decode('ascii', errors='ignore'))
                except:
                    print "pbm d'encodage avec la phrase :  ",
                    print a
        self.nb_logs = len(self.logs)
    def __del__(self):
        self.write_to_disk()
        

In [4]:
import random

In [5]:
class Cycliste:

    def __init__(self, parameters):
        import time
        import random
        from scipy.stats import norm
        import random
        """ ==========================================================
                                    Variables
        ==============================================================="""
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        
        if   age < 35 : facteur = norm( loc =1.2, scale=0.2 )
        elif age < 50 : facteur = norm( loc =1  , scale=0.2 )
        elif age < 90 : facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base  = 20
            nb_km            = norm( loc =7, scale=4 )
        else:
            vitesse_de_base  = 15
            nb_km            = norm( loc =10, scale=4 )
        vitesse_moyenne      = vitesse_de_base * facteur.rvs(1) + sportif
        """ ==========================================================
                                    Attributs
        ==============================================================="""
        self.identifiant     = "cycliste_%s"%parameters[0]
        self.station_maison  = parameters[1]
        self.station_travail = parameters[2]
        self.sportif         = sportif
        self.age             = age
        self.sexe            = sexe
        self.nb_km           = nb_km.rvs(1)[0]
        self.vitesse         = vitesse_moyenne[0]
        self.sur_velo        = False
        self.debug           = False
        self.nb_trajet       = 0
        self.durees          = []
        self.trajets         = []
        self.velo            = False
        self.prises          = []
        self.attente         = 0.5 ; #random.randint(0,10)
        self.last_rendu      = time.time()
        self.logger_cycliste = logger("./../logs/cycliste_cyclistes.csv", 0, header = "cycliste,sportif,age,sexe,nb_km,vitesse,attente"        )
        self.logger_prise    = logger("./../logs/cycliste_prises.csv", header = "cycliste,heure_de_depart,velo,action")
        self.logger_rendu    = logger("./../logs/cycliste_rendu.csv", header = "cycliste,heure_de_depart,heure_d_arrivee,nb_km_trajet,velo.nom")
        self.logger_debug    = logger("./../logs/cycliste_debug.csv",1, header = "cycliste,n_message,time,message")
        
        
        
        
        self.a_la_maison     = True
        self.n_message       = 0
        self.trajet_courant  = ""
        """ ==========================================================
                                    Logs
        ==============================================================="""        
        self.log_cycliste()
        self.logger_debug.log("cree")        
        self.log_debug(u"station_maison = %s"%self.station_maison)
        self.log_debug(u"station_travail = %s"%self.station_travail)
        self.log_debug(u"station_maison nearest= %s"%"|".join([x.nom for x in self.station_maison.nearest]))
        self.log_debug(u"station_travail nearest= %s"%"|".join([x.nom for x in self.station_travail.nearest]))
        
    def peut_prendre_velo(self, station):
        """
        Renvoie un velo ou bien False.
        """
        from time import time
        import random
        """ __________________________________________________________
                            Préconditions
        ______________________________________________________________"""        
        if time() < self.attente + self.last_rendu:
            self.log_debug(u"trop tot de %s ms"% (self.attente + self.last_rendu - time() ))
            return False, False
        if self.sur_velo:
            self.log_debug(u"en route")
            return False, False
        """ __________________________________________________________
                    Recherche de vélo sur les stations proches
        ______________________________________________________________"""        
        random.shuffle(station.nearest)
        for local_station in station.nearest:
            velo = local_station.prendre_velo()            
            if not velo: 
                self.log_debug(u"pas de velo sur la station %s " % local_station)
                continue
            if local_station in self.station_maison.nearest : elem_de = "station maison"
            else                                            : elem_de = "station travail"
            self.log_debug(u"velo pris sur station %s (elem de %s)" % (local_station,elem_de))
            return velo , local_station
        """ __________________________________________________________
                                    Aucun vélo trouvé
        ______________________________________________________________"""        
        self.log_debug(u"Aucun velo trouve aux alentours.")
        return False, False
    
    
    def prend_velo(self):
        import time
        
        if   self.a_la_maison : station = self.station_maison
        else                  : station = self.station_travail
        
        velo, station = self.peut_prendre_velo(station)
        if not velo :
            return 
                       
        self.log_debug(u"%s a pris le velo %s sur la station %s"%(self.a_la_maison, velo, station))
        velo.disponible      = False
        self.sur_velo        = True
        self.heure_de_depart = time.time()
        self.velo            = velo
        self.trajet_courant  = trajet(self, velo,station)
        self.log_prise()
        try:
            nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
        except Exception as e:
            self.log_debug(u"pbm sur nb_km_local = self.nb_km = %s " %self.nb_km)
            nb_km_local = 15
        try:
            vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
            vitesse_local        = vitesse_local * velo.performance
        except:
            self.log_debug(u"pbm sur vitesse_local self.vitesse = %s"%self.vitesse)
            vitesse_local = self.vitesse
            
        self.nb_km_trajet    = nb_km_local
        duree                = nb_km_local / vitesse_local
        self.heure_d_arrivee = self.heure_de_depart + duree
        
        if velo.performance <0.1:
            duree                = 0
            #self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart
            self.nb_km_trajet    = 0
            self.log_debug(u"velo defaillant <0.1, rendu a la meme station :-( ")
            self.rendre_vite_velo(station)
            return
        
        self.a_la_maison = not self.a_la_maison
        if velo.performance <0.3:
            duree                = duree / 10
            #self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart + duree
            self.nb_km_trajet    = self.nb_km_trajet / float(10)
            self.log_debug(u"velo defaillant <0.3, rendu a la meme station :-( ")
            self.rendre_vite_velo(station)
        elif velo.performance <0.5:
            duree                = duree *1.5
            #self.durees.append( duree)
            self.nb_km_trajet    = self.nb_km_trajet 
            self.heure_d_arrivee =  self.heure_de_depart + duree 
            self.log_debug(u"velo defaillant <0.5")
        else:
            pass
            #self.durees.append( duree)




    def rendre_vite_velo(self, station):        
        import time
        
        if not self.sur_velo:
            self.log_debug(u"pas en chemin")    
            return 0
        
        if time.time() < self.heure_d_arrivee:
            self.log_debug(u"pas encore arrive")
            return
        
        if self.a_la_maison:
            station = self.station_maison
            elem_de = "station_maison"
        else:
            station = self.station_travail
            elem_de = "station_travail"
        self.log_debug(u"self.a_la_maison = %s"%self.a_la_maison)    
        
        rendu = False
        for local_station in station.nearest:
            if local_station.rendre_velo(self.velo, self.nb_km_trajet):
                self.log_debug(u"velo rendu sur la station %s ( a la maison = %s) (elem de %s)"%(local_station, self.a_la_maison, elem_de))    
                rendu = True
                break
            else:
                self.log_debug(u"impossible de rendre sur ,%s ( a la maison = %s)"%(local_station, self.a_la_maison))    
        if not rendu:
            return
                        
        self.last_rendu = time.time()
        duree_constatee = self.last_rendu - self.heure_de_depart
        trajet          = (self.heure_de_depart, self.heure_d_arrivee, self.identifiant, self.nb_km_trajet, self.velo.nom )
        self.sur_velo  = False
        self.nb_trajet = self.nb_trajet + 1
        self.log_rendu()
        self.trajet_courant.fin(local_station)
        self.velo      = False
        
        #self.trajets.append(trajet)
    def rendre_velo(self):        
        import time
        
        if not self.sur_velo:
            self.log_debug(u"pas en chemin")    
            return 0
        
        if time.time() < self.heure_d_arrivee:
            self.log_debug(u"pas encore arrive")
            return
        
        if self.a_la_maison:
            station = self.station_maison
            elem_de = "station_maison"
        else:
            station = self.station_travail
            elem_de = "station_travail"
        self.log_debug(u"self.a_la_maison = %s"%self.a_la_maison)    
        
        rendu = False
        for local_station in station.nearest:
            if local_station.rendre_velo(self.velo, self.nb_km_trajet):
                self.log_debug(u"velo rendu sur la station %s ( a la maison = %s) (elem de %s)"%(local_station, self.a_la_maison, elem_de))    
                rendu = True
                break
            else:
                self.log_debug(u"impossible de rendre sur ,%s ( a la maison = %s)"%(local_station, self.a_la_maison))    
        if not rendu:
            return
                        
        self.last_rendu = time.time()
        duree_constatee = self.last_rendu - self.heure_de_depart
        trajet          = (self.heure_de_depart, self.heure_d_arrivee, self.identifiant, self.nb_km_trajet, self.velo.nom )
        self.sur_velo  = False
        self.nb_trajet = self.nb_trajet + 1
        self.log_rendu()
        self.trajet_courant.fin(local_station)
        self.velo      = False
        
        #self.trajets.append(trajet)
        
            
    def __str__(self):
        return self.identifiant
    def log_cycliste(self):

        to_print = u"%s,%s,%s,%s,%s,%s,%s"%(    self.identifiant   ,self.sportif     ,
                                                 self.age         ,self.sexe        ,
                                                 self.nb_km       ,self.vitesse     , self.attente )
        
        self.logger_cycliste.log(to_print)
            
    def log_prise(self, action="prise"):

        to_print = u"%s,%s,%s,%s"%( self.identifiant,self.heure_de_depart, self.velo.nom, action)
        self.logger_prise.log(to_print)
            
    def log_rendu(self, action="rendu"):

        to_print = u"%s,%s,%s,%s,%s\n"%( self.identifiant, self.heure_de_depart, 
                                       self.heure_d_arrivee,  self.nb_km_trajet, self.velo.nom )

        self.logger_rendu.log(to_print)
        
    def log_debug(self, message):
        from time import time
        to_print = u"%s,%04d, %s,%s"% (self.identifiant,self.n_message, time() , message)
        self.n_message = self.n_message + 1
        self.logger_debug.log(to_print)
    
        


In [6]:
class trajet():
    def __init__(self, cycliste, velo, station):
        from time import time
        self.cycliste        = cycliste
        self.velo            = velo
        self.station_depart  = station
        self.heure_de_depart = time()
        champs = ["station_depart", "cycliste", "velo", "heure_de_depart", "heure_de_fin", "duree", "distance", "station_arrivee"]
        self.logger          = logger("./../logs/trajets.csv", header =",".join(champs))
        
    def fin(self, station):
        self.heure_de_fin     = time()
        self.duree            = self.heure_de_fin - self.heure_de_depart
        self.station_arrivee  = station
        A = self.station_depart.position
        B = self.station_arrivee.position
        self.distance = math.sqrt( (A[0]-B[0])**2 + (A[1]-B[1])**2 )
        champs = ["station_depart", "cycliste", "velo", "heure_de_depart", "heure_de_fin", "duree", "distance", "station_arrivee"]
        to_print = ",".join(["%s"%self.__dict__[x] for x in champs])
        self.logger.log(to_print)


# -1) Velo

In [7]:
class Velo:

    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        performance            = norm( loc =0.9, scale=0.2 )
        
        self.nom               = "%s_%s"%(nom, time.time())
        self.performance       = performance.rvs(1)[0]
        self.station           = station
        self.degradation       = 0
        self.performances      = []
        self.debug             = False
        self.nb_km_trajet      = 0
        self.heures_rendu      = []
        self.disponible        = True
        self.logger_reparation = logger("./../logs/velo_reparations.csv", header = "velo,n_message,time,performance")
        self.logger_etat       = logger("./../logs/velos_etats.csv", header = "velo,n_message,time,station.nom,performance,nb_km_trajet")
                
        

        self.probleme_list     = self.get_problemes_list()
        self.n_message         = -1

    def get_problemes_list(self):
        """
        Créé le tableau de dégradations possible pour un vélo.
        """
        import math
        problemes       = [0] * int(math.ceil((50*self.performance)))
        problemes_bis   = {u"pedale" : 0.2, u"roue"  : 1 , u"degonfle" : 0.3, u"freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.values())
        return problemes
        
    def log_reparation(self):
        import time
        self.n_message = self.n_message + 1
        reparation     = (self.nom,self.n_message, time.time(),self.performance)
        to_print       = u"%s,%s, %s,%s"%reparation
        self.logger_reparation.log(to_print)
        

    def log_etat(self):
        from time import time
        self.n_message = self.n_message + 1
        to_print = u"%s,%s,%s,%s,%s,%s\n"%(   self.nom           ,
                                            self.n_message,
                                           time()            ,  
                                           self.station.nom  ,
                                           self.performance  ,
                                           self.nb_km_trajet )
        self.logger_etat.log(to_print)
    
    def rendu(self, station, nb_km_trajet):
        import time
        degradation       = random.choice(self.probleme_list)        
        self.disponible   = True
        self.performance  = self.performance - self.performance*degradation                                        
        self.nb_km_trajet = nb_km_trajet
        self.station      = station
        self.log_etat()
        if 900 < random.randint(0, 1000 ) :
            self.performance  = 1
            self.log_reparation()
        
        

    def __str__(self):
        return "%s"%self.nom



# -1 ) Station

In [8]:
class Station:
    def __init__(self, parameters, debug=False ):
        
        import random
        import time
        
        self.nom               = parameters[0]
        self.position          = parameters[1]
        self.nb_plot           = random.choice([10,15,20, 30])
        self.nb_libre          = 0 
        self.nb_velos          = self.nb_plot
        self.velos             = []        
        self.n_message         = -1
        self.debug             = False
        self.avaries           = []
        self.arrete            = False
        self.liste_d_avaries   = []
        self.nb_fois_en_arret  = 0
        self.logger_dispos     = logger("./../logs/stations_dispos.csv", header = "nom,n_message,time,nb_plot,nb_libre,nb_velos")
        self.logger_reparation = logger("./../logs/stations_reparations.csv")
        self.logger_avarie     = logger("./../logs/stations_avarie.csv", header = "nom,n_message,time,avarie")
        
        self.log_dispo()

        noms = get_names()
        for n in range(self.nb_velos ):
            nom_velo= u"velo_"+"".join(["%s"%x for x in random.choice(noms)])
            self.velos.append(Velo(nom_velo , self))
        

    def log_reparation(self):
        self.log_dispo()
            
            
    def log_dispo(self):
        from time import time
        self.n_message = self.n_message + 1
        dispo =(self.nom,self.n_message, time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s,%s"%dispo
        
        self.logger_dispos.log(to_print)
            
    def log_avarie(self, avarie):
        from time import time
        self.n_message = self.n_message + 1
        to_print = u"%s,%s,%s,%s"%(self.nom,self.n_message, time(), avarie) 
        
        self.logger_avarie.log(to_print)

    
    def avarie_possible(self):
        import math
        import random
        import time
        problemes   = { "plot"       : range(0, 20) ,
                       "half"        : range(20,30) ,
                       "network"     : range(30,35) , 
                       "electricite" : range(35,60) , 
                       "bug"         : range(60,90) }
        zero     = 0
        mil      = 1000
        cent_mil = 100 * mil 
        
        for k,v in problemes.iteritems():
            if k in self.liste_d_avaries:
                r = random.randint(mil, cent_mil)
                new_range = range(r, r+30)
                problemes[k].extend(new_range)
        
        alea = random.randint(zero,cent_mil)
        avarie = False
        for k,v in problemes.iteritems():
            if alea in v:
                avarie = k
                
        if avarie:
            self.avaries.extend([avarie])
            if   avarie == "plot" : self.nb_libre = self.nb_libre -1
            elif avarie == "half" : self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
            elif avarie == "bug"  : self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
            elif avarie == "network" or avarie == "electricite":
                self.nb_libre = 0
                self.arrete = True
                
            if self.nb_libre <0:
                self.nb_libre = 0            
            self.log_dispo()

            if avarie:
                self.log_avarie("%s"%avarie)
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.log_dispo()
            return 1
        if 9500 < random.randint(0, 10000 ) :
            self.reparer()
        return 0
    
    def reparer(self):
        self.nb_libre = self.nb_plot - self.nb_velos
        self.arrete   = False
        self.log_reparation()
        
    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.log_dispo()
            return to_return
        
        if self.arrete:
            self.nb_fois_en_arret += 1
        
        if self.nb_fois_en_arret > 10:
            self.reparer()

        return 0
    def get_nearest_station(self):
        x = self.position[0]
        y = self.position[1]
        deplacement_x = [-1, 0, 1]
        deplacement_y = [-1, 0, 1]
        deplacements  = [ (x_proche , y_proche) for x_proche in deplacement_x for y_proche in  deplacement_y if x_proche or y_proche]
        positions_proches = [(x +  a , y +  b) for a,b in  deplacements ]
        nearests = []
        for x2, y2 in positions_proches:
            if x2 + x < 0 or y2 + y < 0:
                continue 
            position_la_plus_proche = x2 * cote_len  + y2
            if 0 <= position_la_plus_proche and position_la_plus_proche < len(stations):
                nearests.append(stations[position_la_plus_proche])
        return nearests
        
                
    def __str__(self):
        return u"%s"%(self.nom)


# 0) get_names

In [9]:
def get_names(nb_lettres=5, nb_noms = 15000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        nb_noms = nb_noms -1
        if nb_noms <2:
            break
    shuffle(names)
    return names

# 0) création des stations

In [10]:
import math

In [11]:
deplacement_x = [-1, 0, 1]
deplacement_y = [-1, 0, 1]

In [12]:
from time import time
import multiprocessing
nb_cpu       = 4
nb_station   = 100
debut        = time()
process_pool = multiprocessing.Pool(nb_cpu)
debut        = time()
names        = get_names()
cote_len     = int(math.ceil(math.sqrt(nb_station)))
positions    = [ (longitude, lattitude) for longitude in range(cote_len) for lattitude in range(cote_len)]
parameters   = [(names[i], positions[i]) for i in range(nb_station)]

stations     = [Station(p) for p in parameters]
for x in stations:
    x.nearest = x.get_nearest_station()
nb_velos     = sum([len(station.velos) for station in stations])
duree = time() - debut
print "duree = %s pour %s cree soit %s sec / unite"%(duree, nb_velos, duree/nb_velos)


duree = 3.10994696617 pour 1775 cree soit 0.00175208279784 sec / unite


# Creation des cycliste

In [13]:
aller_a = []
probas  = {}
n = -1
for station in stations:
    n           = n + 1
    milieu      =  cote_len / 2 
    centre      = (milieu, milieu)
    adjacent_1  = (station.position[0] - milieu)
    adjacent_2  = (station.position[1] - milieu)
    hypothenuse = math.sqrt( adjacent_1*adjacent_1 + adjacent_2 *adjacent_2 ) 
    probas      = int(math.ceil(milieu - math.floor(hypothenuse) + 2))
    aller_a.extend([station]*probas)    

In [14]:
names        = get_names(5, nb_velos*11)

In [15]:
debut        = time()
n            = 4
multiple     = nb_velos*2
process_pool = multiprocessing.Pool(n)
names        = get_names(5, multiple)
noms         = [(names[i], random.choice(stations), random.choice(aller_a)) for i in range(multiple-1)]
cyclistes    = [Cycliste(noms[i]) for i in range(int(math.floor(nb_velos*1.3)) )]
duree        = time() - debut
print "duree = %s pour %s cree soit %s sec / unite"%(duree, len(cyclistes), duree/nb_velos)


duree = 5.09338307381 pour 2307 cree soit 0.00286951159088 sec / unite


In [16]:
velos = [x.velos for x in stations]

# voyages

In [17]:
if False:
    to_fetch = ["nb_plot", "nb_libre", "nb_velos", "position"]
    for station in stations[:2]:
        for item in to_fetch:
            print item, station.__dict__[item]


In [18]:
if False:
    to_fetch = ["disponible"] #, "nb_km_trajet"]
    for velo in velos:
        data = [x.__dict__[info] for x in velo for info in to_fetch  ]
        print data

In [19]:
from time import time
import multiprocessing
import math
from time import time

In [ ]:
from time import gmtime, strftime

In [ ]:
depart = time()
print "debut a ", strftime("%Y-%m-%d %H:%M:%S", gmtime())
n = 0  
a         = time()
while time() < depart + 10000 :     
    old_nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
    en_route = filter(lambda cycliste:     cycliste.sur_velo, cyclistes)
    a_pied   = filter(lambda cycliste: not cycliste.sur_velo, cyclistes)
    
    map(lambda cycliste: cycliste.rendre_velo(), en_route  )
    map(lambda cycliste: cycliste.prend_velo(),  a_pied    )
    
    new_nb_trajet =  sum([cycliste.nb_trajet for cycliste in cyclistes]) - old_nb_trajet
    
    if n%10 ==0:
        duree_totale = time()-a
        nb_arrete = sum([ s.__dict__['arrete'] for s in stations ])
        print "(tour n°%03d)%03d/% 3.2f| %s "%(n, new_nb_trajet, duree_totale, nb_arrete)
        a         = time()

    n += 1
    
print "*"*30
print "fini"
print "*"*30
print "fin a ", strftime("%Y-%m-%d %H:%M:%S", gmtime())

debut a  2017-02-21 15:19:10
(tour n°000)000/ 3.89| 0 


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:136: RuntimeWarning: divide by zero encountered in double_scalars


(tour n°010)1139/ 26.38| 1 
(tour n°020)1147/ 27.99| 0 
(tour n°030)1131/ 28.13| 0 
(tour n°040)1203/ 27.92| 0 
(tour n°050)1218/ 28.77| 0 
(tour n°060)1138/ 28.14| 0 
(tour n°070)1176/ 27.39| 0 
(tour n°080)1180/ 27.78| 0 
(tour n°090)1201/ 27.61| 0 
(tour n°100)1131/ 28.74| 1 
(tour n°110)1148/ 27.82| 0 
(tour n°120)1174/ 27.60| 1 
(tour n°130)1227/ 27.37| 0 
(tour n°140)1185/ 31.19| 0 
(tour n°150)1187/ 37.12| 0 
(tour n°160)1233/ 40.08| 0 
(tour n°170)1184/ 45.51| 0 
(tour n°180)1172/ 38.61| 0 
(tour n°190)1130/ 33.89| 0 
(tour n°200)1140/ 37.12| 0 
(tour n°210)1125/ 42.03| 1 
(tour n°220)1157/ 37.53| 0 
(tour n°230)1209/ 32.04| 0 
(tour n°240)1176/ 30.06| 0 
(tour n°250)1199/ 30.06| 0 
(tour n°260)1161/ 30.78| 1 
(tour n°270)1195/ 31.80| 0 
(tour n°280)1182/ 31.16| 0 
(tour n°290)1218/ 31.65| 0 
(tour n°300)1155/ 30.22| 0 
(tour n°310)1128/ 30.98| 0 
(tour n°320)1104/ 30.23| 0 
(tour n°330)1204/ 30.68| 0 
(tour n°340)1120/ 30.34| 0 
(tour n°350)1158/ 30.15| 0 
(tour n°360)1145/ 29

In [ ]:
print "fini 2"

In [ ]:
save_network = True
if save_network:
    import pickle
    path_to_save_stations = "./../logs/stations.csv"
    with open(path_to_save_stations, "w") as output_file:
        pickle.dump(stations, output_file)
    path_to_save_cyclistes = "./../logs/cyclistes.csv"
    with open(path_to_save_cyclistes, "w") as output_file:
        pickle.dump(cyclistes, output_file)

In [ ]:
if False :
    load_network   = False
    nb_de_secondes = 1800
    try:
        if load_network:
            stations2  = pickle.load(open(path_to_save_stations ))
            cyclistes2 = pickle.load(open(path_to_save_cyclistes))
        depart     = time()
        print "debut a ", strftime("%Y-%m-%d %H:%M:%S", gmtime())
        n = 0  
        a = time()
        while time() < depart + nb_de_secondes :     
            old_nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes2])
            en_route = filter(lambda cycliste:     cycliste.sur_velo, cyclistes2)
            a_pied   = filter(lambda cycliste: not cycliste.sur_velo, cyclistes2)

            map(lambda cycliste: cycliste.rendre_velo(), en_route  )
            map(lambda cycliste: cycliste.prend_velo(),  a_pied    )

            new_nb_trajet =  sum([cycliste.nb_trajet for cycliste in cyclistes2]) - old_nb_trajet

            if n%10 ==0:
                duree_totale = time()-a
                nb_arrete = sum([ s.__dict__['arrete'] for s in stations2 ])
                print "(tour n°%03d)%03d/% 3.2f| %s "%(n, new_nb_trajet, duree_totale, nb_arrete)
                a = time()

            n += 1

        print "*"*30
        print "fini"
        print "*"*30
        print "fin a ", strftime("%Y-%m-%d %H:%M:%S", gmtime())
    except:
        print "on enregistre la situation"
        import pickle
        path_to_save_stations = "./../logs/stations_%s.csv"%time()
        with open(path_to_save_stations, "w") as output_file:
            pickle.dump(stations, output_file)
        path_to_save_cyclistes = "./../logs/cyclistes_%s.csv"%time()
        with open(path_to_save_cyclistes, "w") as output_file:
            pickle.dump(cyclistes, output_file)    